In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
names = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('u.item', sep='|', names=names,encoding='latin-1')

In [4]:
col_names = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('u1.base', sep='\t', names=col_names, encoding='latin-1')
ratings_test = pd.read_csv('u1.test', sep='\t', names=col_names, encoding='latin-1')


In [5]:
ratings_base.drop('unix_timestamp',axis=1,inplace=True)

In [6]:
ratings_test.drop('unix_timestamp',axis=1,inplace=True)

In [7]:
movie_data = items.filter(['movie id','movie title'],axis=1)

In [8]:
ratings_base.head()

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [9]:
movie_data.isna().sum()

movie id       0
movie title    0
dtype: int64

In [10]:
ratings_base.isna().sum()

user_id     0
movie_id    0
rating      0
dtype: int64

In [11]:
ratings_test.isna().sum()

user_id     0
movie_id    0
rating      0
dtype: int64

In [12]:
movie_data.rename(columns={'movie title':'title','movie id':'movie_id'},inplace=True)

In [14]:
user_item_pivot = ratings_base.pivot('user_id','movie_id','rating').fillna(0)

In [15]:
user_item_pivot.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
user_similarity = cosine_similarity(user_item_pivot)

In [38]:
def movie_recommender(user_item_pivot, user_similarity, user, k=20, top=10):
    # Get location of the actual movie in the User-Items matrix
    uindex = user_item_pivot.index.get_loc(user)
    # Use it to index the User similarity matrix
    similar_users = user_similarity[uindex]
    # obtain the indices of the top k most similar users
    similar_users = user_item_pivot.index[similar_users.argpartition(-k)[-k:]]
    # Obtain the mean ratings of those users for all movies
    movies = user_item_pivot.loc[similar_users].mean(0).sort_values(ascending=False)
    # Discard already seen movies
    seen_movies = user_item_pivot.loc[user].gt(0)
    seen_movies = seen_movies.index[seen_movies].tolist()
    #top_movies = movies.drop(seen_movies).head(top)
    top_movies = movies.drop(seen_movies)
    top_movies = pd.DataFrame(top_movies).reset_index().rename(columns={0:'Mean_Rating'})
    # return recommendations - top similar users rated movies
    #return top_movies.index.to_frame().reset_index(drop=True)
    return top_movies.merge(movie_data, how='inner', on='movie_id')

def test_recommendation(top_movies,user):
    unseen = ratings_test[ratings_test['user_id']==user]
    actual = unseen[unseen['movie_id'].isin(top_movies['movie_id'])]['rating']
    squared_error = (top_movies['Mean_Rating'] - actual)**2
    mse = np.nanmean(squared_error)
    return mse

In [48]:
rec = movie_recommender(user_item_pivot,user_similarity,1)

In [49]:
mse = test_recommendation(rec,1)

In [50]:
np.sqrt(mse)

1.900048020906794

In [32]:
rec.head(10)

,movie_id,Mean_Rating,title
0,174,4.30,Raiders of the Lost Ark (1981)
1,96,4.20,Terminator 2: Judgment Day (1991)
2,98,4.10,"Silence of the Lambs, The (1991)"
3,12,4.00,"Usual Suspects, The (1995)"
4,100,3.90,Fargo (1996)
5,56,3.85,Pulp Fiction (1994)
6,69,3.75,Forrest Gump (1994)
7,64,3.60,"Shawshank Redemption, The (1994)"
8,423,3.60,E.T. the Extra-Terrestrial (1982)
9,183,3.35,Alien (1979)


PEARSON COEFFICIENT

In [42]:
pearson_data = ratings_base.pivot('movie_id','user_id','rating').fillna(0)

In [43]:
pearson_similarities = pearson_data.corr(method='pearson').to_numpy()

In [44]:
pearson_result = movie_recommender(ratings_base.pivot('user_id','movie_id','rating').fillna(0),pearson_similarities,11)

In [45]:
pearson_result.head(10)

,movie_id,Mean_Rating,title
0,50,4.60,Star Wars (1977)
1,174,4.15,Raiders of the Lost Ark (1981)
2,204,4.00,Back to the Future (1985)
3,172,3.90,"Empire Strikes Back, The (1980)"
4,181,3.75,Return of the Jedi (1983)
5,216,3.40,When Harry Met Sally... (1989)
6,22,3.40,Braveheart (1995)
7,64,3.35,"Shawshank Redemption, The (1994)"
8,96,3.15,Terminator 2: Judgment Day (1991)
9,191,3.10,Amadeus (1984)


In [46]:
mse_pearson = test_recommendation(pearson_result,1)

In [51]:
np.sqrt(mse_pearson)

1.9088413275270457